# Using `pyfwg` for direct morphing with `morph_epw_global` and `morph_epw_europe`

This notebook demonstrates the simplest and most direct way to use the `pyfwg` library: the `morph_epw_global` and `morph_epw_europe` functions.

These functions are designed as one-shot tools that provide full access to all of the Future Weather Generator's parameters without requiring the user to manage the multi-step `MorphingWorkflow` class. It handles parameter validation, temporary file management, and final file placement in a single call.

## Understanding the Parameters

The `morph_epw_global` and `morph_epw_europe` functions are powerful because these exposes all of the underlying tool's options. Here is a complete reference for all their arguments.

### Core Parameters

*   `epw_paths` (`Union[str, List[str]]`): **Required.** The path to a single EPW file (as a string) or a list of paths to multiple EPW files.
*   `fwg_jar_path` (`str`): **Required.** The absolute or relative path to the `FutureWeatherGenerator_v3.0.0.jar` file.

### Workflow Control Parameters

*   `output_dir` (`str`, default: `'./morphed_epws'`): The directory where the final morphed `.epw` and `.stat` files will be saved.
*   `delete_temp_files` (`bool`, default: `True`): If `True`, the temporary folders are deleted after processing. Set to `False` for debugging.
*   `temp_base_dir` (`str`, default: `'./morphing_temp_results'`): The base directory where temporary folders will be created.
*   `fwg_show_tool_output` (`bool`, default: `False`): If `True`, prints the FWG tool's console output in real-time. Highly recommended for monitoring progress.
*   `fwg_params` (`Optional[Dict]`, default: `None`): A dictionary for providing a base set of FWG parameters. Any explicit `fwg_` argument will override the values in this dictionary.

### Future Weather Generator Tool Parameters

These arguments correspond directly to the options in the FWG command-line tool.

*   `fwg_gcms` (`Optional[List[str]]`, default: `None`): **Only valid for `morph_epw_global`**. A list of GCMs to use. If `None`, the full default list is used to create an ensemble.
*   `fwg_rcm_pairs` (`Optional[List[str]]`, default: `None`): **Only valid for `morph_epw_europe`**. List of GCM-RCM pairs to use. If None, the tool's default list is used.
*   `fwg_create_ensemble` (`bool`, default: `True`): If `True`, creates an ensemble from the selected GCMs (Argument: `%ensemble%`).
*   `fwg_winter_sd_shift` (`float`, default: `0.0`): Temperature's standard deviation shift for the winter peak month. Range: -2.0 to 2.0.
*   `fwg_summer_sd_shift` (`float`, default: `0.0`): Temperature's standard deviation shift for the summer peak month. Range: -2.0 to 2.0.
*   `fwg_month_transition_hours` (`int`, default: `72`): Number of hours to smooth the transition between months. Range: 0 to 336.
*   `fwg_use_multithreading` (`bool`, default: `True`): If `True`, performs computations in multithread (Argument: `%do_multithred_computation%`).
*   `fwg_interpolation_method_id` (`int`, default: `0`): Selects the grid method. Options: `0` (inversed distance weighting), `1` (average of four nearest points), `2` (nearest point).
*   `fwg_limit_variables` (`bool`, default: `True`): If `True`, bounds each generated variable to its physical limits (Argument: `%do_limit_variables%`).
*   `fwg_solar_hour_adjustment` (`int`, default: `1`): Option for solar hour adjustment. Options: `0` (None), `1` (ByMonth), `2` (ByDay).
*   `fwg_diffuse_irradiation_model` (`int`, default: `1`): Option for diffuse solar irradiation model. Options: `0` (Ridley), `1` (Engerer), `2` (Paulescu).
*   `fwg_add_uhi` (`bool`, default: `True`): Option to pre-process the Urban Heat Island (UHI) effect.
*   `fwg_epw_original_lcz` (`int`, default: `14`): The Local Climate Zone (LCZ) of the original EPW. Range: 1 to 17.
*   `fwg_target_uhi_lcz` (`int`, default: `1`): The target LCZ of the generated EPW. Range: 1 to 17.

## Using `morph_epw_global`

### Step 1: Imports and Setup

First, we'll import the necessary function and set up our file paths. 

**Important:** You must change the `jar_path` variable to the correct location of the `FutureWeatherGenerator_v3.0.1.jar` file on your system. You must also ensure that the `epw_file` path is correct relative to where you are running this notebook.

In [1]:
import os
from pyfwg import morph_epw_global, get_available_lczs, check_lcz_availability, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# Define the path to the EPW file to be processed. You can also define a list of paths to process multiple EPW files.
epw_file = 'epws/w_pattern/sevilla_uhi-type-1.epw'

# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar"

### Step 2: Pre-flight Checks (Recommended). Check Available LCZs.

Before running a long morphing process, it's a good practice to validate your parameters. `pyfwg` provides utility functions to help with this.

The `fwg_epw_original_lcz` and `fwg_target_uhi_lcz` parameters depend on which Local Climate Zones are available for a specific weather file location. We can find this out using `get_available_lczs`.

In [2]:
available_zones = get_available_lczs(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    # In java_class_path_prefix, use 'futureweathergenerator' for the global scope FWG tool, otherwise 'futureweathergenerator_europe'
    java_class_path_prefix='futureweathergenerator',
    show_tool_output=False,
)

2025-09-25 09:32:01 - INFO - --- Fetching available LCZs for 1 EPW file(s) ---
2025-09-25 09:32:01 - INFO - Checking LCZ pair (Original: 0, Target: 0) availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:01 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-09-25 09:32:01 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.UHI_Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw C:\Users\sanga\AppData\Local\Temp\tmp0mapkcd5/ true 0:0
2025-09-25 09:32:07 - INFO - Available LCZs for 'sevilla_uhi-type-1.epw': [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


In [3]:
# Print the results in a user-friendly format
for filename, lcz_list in available_zones.items():
    print(f"\nAvailable LCZs for: {filename}")
    if lcz_list:
        print(f"  {lcz_list}")
    else:
        print("  Could not be determined.")


Available LCZs for: sevilla_uhi-type-1.epw
  [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


### Step 2: Run the Morphing Process

Now, we can call the `morph_epw_global` function. We will override a few of the default parameters to demonstrate how to customize the process. To speed up the process, let's use just one GCM. If you don't know which are these, you can import the list containing the default models:

In [4]:
from pyfwg import DEFAULT_GLOBAL_GCMS
print(DEFAULT_GLOBAL_GCMS)

{'CMCC_ESM2', 'MRI_ESM2_0', 'CanESM5_1', 'EC_Earth3', 'CNRM_ESM2_1', 'CAS_ESM2_0', 'EC_Earth3_Veg', 'CNRM_CM6_1', 'MIROC_ES2L', 'CanESM5_CanOE', 'EC_Earth3_Veg_LR', 'CanESM5', 'UKESM1_0_LL', 'IPSL_CM6A_LR', 'GISS_E2_1_H', 'MIROC_ES2H', 'CNRM_CM6_1_HR', 'GFDL_ESM4', 'FGOALS_g3', 'BCC_CSM2_MR', 'GISS_E2_2_G', 'MIROC6', 'GISS_E2_1_G'}


So let's use for instance "CanESM5":

In [5]:
# The validation of parameters is executed automatically inside the function.
created_files_custom = morph_epw_global(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    output_dir='./custom_output_global',
    fwg_show_tool_output=True,      # See the tool's progress in real-time
    fwg_gcms=['CanESM5'],           # Use only one specific GCM for speed
    fwg_interpolation_method_id=2,  # Use the "nearest point" method
    fwg_epw_original_lcz=2,         # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3          # Use a validated LCZ from the check above
)

2025-09-25 09:32:07 - INFO - --- Starting Direct Global Morphing Process ---
2025-09-25 09:32:07 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-09-25 09:32:07 - INFO - Validating LCZ availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:07 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:07 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-09-25 09:32:07 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.UHI_Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw C:\Users\sanga\AppData\Local\Temp\tmp9zjocxtm/ true 2:3



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global
  - EPWs to be Morphed (1 files):
    - sevilla_uhi-type-1.epw

  For input file: sevilla_uhi-type-1.epw
    -> Generated 'ssp126_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp126_2050.epw
    -> Generated 'ssp245_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp245_2050.epw
    -> Generated 'ssp370_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp370_2050.epw
    -> Generated 'ssp585_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp585_2050.epw
    -> Generated 'ssp126_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ssp126_2080.epw
    -> Generated 'ssp245_2080.epw' will be moved to: D

2025-09-25 09:32:11 - INFO - UHI effect applied successfully.
2025-09-25 09:32:11 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 09:32:11 - INFO - Copied input file to temporary directory: ./morphing_temp_results\sevilla_uhi-type-1\sevilla_uhi-type-1.epw



-------------------- Executing FWG for sevilla_uhi-type-1.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw CanESM5 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\sevilla_uhi-type-1/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 09:32:56 - INFO - Direct global morphing complete. 17 files created in D:\Python\pyfwg\pyfwg\tutorials\custom_output_global


  --- End of FWG Output ---


In [6]:
print("\nSuccessfully created files:")
for f in created_files_custom:
    print(f)


Successfully created files:
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp126_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp126_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp126_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp126_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp245_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp245_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp245_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp245_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp370_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_global\ESP_-_SEVILLA_Ensemble_ssp370_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_

### Expected Output

When you run the code above, you should see:
1. A detailed printout of the command being executed for your EPW file.
2. The real-time console output from the Future Weather Generator tool (since `fwg_show_tool_output=True`).
3. A final message listing all the `.epw` and `.stat` files that were successfully created and moved to the `./custom_output_global` directory.

Let's delete the files we just generated.

In [7]:
import shutil
shutil.rmtree('custom_output_global')

## Using `morph_epw_europe`

>**FYI**: the usage of `morph_epw_europe` is very similar to the global version. The only differences are:

>* Instead of argument `fwg_gcms`, you need to use `fwg_rcm_pairs`.
>* Instead of variable `DEFAULT_GLOBAL_GCMS`, the available models are in variable `DEFAULT_EUROPE_RCMS`. Therefore, the models you can use in `fwg_rcm_pairs` are those included in `DEFAULT_EUROPE_RCMS`.
>* You have to remember changing the `jar_path` to the FutureWeatherGenerator_Europe_vx.x.x.jar file.

### Step 1: Imports and Setup

First, we'll import the necessary function and set up our file paths. 

**Important:** You must change the `jar_path` variable to the correct location of the `FutureWeatherGenerator_Europe_v1.0.1.jar` file on your system. You must also ensure that the `epw_file` path is correct relative to where you are running this notebook.

In [8]:
import os
from pyfwg import morph_epw_europe

# --- Configuration ---
# Define the path to the EPW file to be processed.
epw_file = 'epws/w_pattern/sevilla_uhi-type-1.epw'

# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar"

### Step 2: Pre-flight Checks (Recommended). Check Available LCZs.

Before running a long morphing process, it's a good practice to validate your parameters. `pyfwg` provides utility functions to help with this.

The `fwg_epw_original_lcz` and `fwg_target_uhi_lcz` parameters depend on which Local Climate Zones are available for a specific weather file location. We can find this out using `get_available_lczs`.

In [9]:
available_zones = get_available_lczs(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    # In java_class_path_prefix, use 'futureweathergenerator' for the global scope FWG tool, otherwise 'futureweathergenerator_europe'
    java_class_path_prefix='futureweathergenerator_europe',
    show_tool_output=False,
)

2025-09-25 09:32:56 - INFO - --- Fetching available LCZs for 1 EPW file(s) ---
2025-09-25 09:32:56 - INFO - Checking LCZ pair (Original: 0, Target: 0) availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:56 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-09-25 09:32:56 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar" futureweathergenerator_europe.UHI_Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw C:\Users\sanga\AppData\Local\Temp\tmpv889plc0/ true 0:0
2025-09-25 09:32:59 - INFO - Available LCZs for 'sevilla_uhi-type-1.epw': [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


In [10]:
# Print the results in a user-friendly format
for filename, lcz_list in available_zones.items():
    print(f"\nAvailable LCZs for: {filename}")
    if lcz_list:
        print(f"  {lcz_list}")
    else:
        print("  Could not be determined.")


Available LCZs for: sevilla_uhi-type-1.epw
  [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


### Step 3: Run the Morphing Process

Now, we can call the `morph_epw_europe` function. We will override a few of the default parameters to demonstrate how to customize the process. To speed up the process, let's use just one RCM. If you don't know which are these, you can import the list containing the default models:

In [11]:
from pyfwg import DEFAULT_EUROPE_RCMS
print(DEFAULT_EUROPE_RCMS)

{'MOHC_HadGEM2_ES_SMHI_RCA4', 'MPI_M_MPI_ESM_LR_SMHI_RCA4', 'CNRM_CERFACS_CNRM_CM5_CNRM_ALADIN63', 'ICHEC_EC_EARTH_SMHI_RCA4', 'NCC_NorESM1_M_SMHI_RCA4', 'MOHC_HadGEM2_ES_DMI_HIRHAM5', 'ICHEC_EC_EARTH_DMI_HIRHAM5'}


So let's use for instance "ICHEC_EC_EARTH_SMHI_RCA4":

In [12]:
# The validation of parameters is executed automatically inside the function.
created_files_custom = morph_epw_europe(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    output_dir='./custom_output_europe',
    fwg_show_tool_output=True,      # See the tool's progress in real-time
    fwg_rcm_pairs=['ICHEC_EC_EARTH_SMHI_RCA4'],           # Use only one specific GCM for speed
    fwg_interpolation_method_id=2,  # Use the "nearest point" method
    fwg_epw_original_lcz=2,         # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3          # Use a validated LCZ from the check above
)

2025-09-25 09:32:59 - INFO - --- Starting Europe-Specific Direct Morphing Process ---
2025-09-25 09:32:59 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-09-25 09:32:59 - INFO - Validating LCZ availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:59 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for sevilla_uhi-type-1.epw...
2025-09-25 09:32:59 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-09-25 09:32:59 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar" futureweathergenerator_europe.UHI_Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw C:\Users\sanga\AppData\Local\Temp\tmpmhbb_9px/ true 2:3



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe
  - EPWs to be Morphed (1 files):
    - sevilla_uhi-type-1.epw

  For input file: sevilla_uhi-type-1.epw
    -> Generated 'rcp26_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp26_2050.epw
    -> Generated 'rcp45_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp45_2050.epw
    -> Generated 'rcp85_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp85_2050.epw
    -> Generated 'rcp26_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp26_2080.epw
    -> Generated 'rcp45_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\rcp45_2080.epw
    -> Generated 'rcp85_2080.epw' will be moved to: D:\Py

2025-09-25 09:33:04 - INFO - UHI effect applied successfully.
2025-09-25 09:33:04 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 09:33:04 - INFO - Copied input file to temporary directory: ./morphing_temp_results_europe\sevilla_uhi-type-1\sevilla_uhi-type-1.epw



-------------------- Executing FWG for sevilla_uhi-type-1.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar" futureweathergenerator_europe.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw ICHEC_EC_EARTH_SMHI_RCA4 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\sevilla_uhi-type-1/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 09:34:03 - INFO - Europe morphing complete. 13 files created in D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe


  --- End of FWG Output ---


In [13]:
print("\nSuccessfully created files:")
for f in created_files_custom:
    print(f)


Successfully created files:
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp26_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp26_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp26_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp26_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp45_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp45_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp45_2080.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp45_2080.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp85_2050.epw
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP_-_SEVILLA_Ensemble_rcp85_2050.stat
D:\Python\pyfwg\pyfwg\tutorials\custom_output_europe\ESP

### Expected Output

When you run the code above, you should see:
1. A detailed printout of the command being executed for your EPW file.
2. The real-time console output from the Future Weather Generator tool (since `fwg_show_tool_output=True`).
3. A final message listing all the `.epw` and `.stat` files that were successfully created and moved to the `./custom_output_europe` directory.

Let's delete the files we just generated.

In [14]:
shutil.rmtree('custom_output_europe')